In [1]:
%pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]
%pip install gym[accept-rom-license]
%pip install pyglet


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gym
import random

C:\Users\BNPC\anaconda3\envs\PAcman\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [3]:
env = gym.make('MsPacman-v0')
#env = gym.make('ALE/MsPacman-v5')#, render_mode='human')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

# sprawdzić evny

In [4]:
env.unwrapped.get_action_meanings()

['NOOP',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT']

In [5]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice(range(actions))
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

C:\Users\BNPC\anaconda3\envs\PAcman\lib\site-packages\gym\envs\atari\environment.py:257: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "
C:\Users\BNPC\anaconda3\envs\PAcman\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Episode:1 Score:190.0
Episode:2 Score:190.0
Episode:3 Score:170.0
Episode:4 Score:260.0
Episode:5 Score:670.0


In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam, SGD

In [7]:
def build_model(height, width, channels, actions):
    model = Sequential(layers=[
        Convolution2D(32, (8, 8), strides = (4,4), activation='relu', input_shape = (4,height,width,channels)),
        Convolution2D(64, (4, 4), strides = (2,2), activation='relu'),
        #Convolution2D(64, (3, 3), activation='relu'),
        Flatten(),
        #Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(actions, activation='linear')])
    print(model.summary())
    return model


In [8]:
(height, width)

(210, 160)

In [21]:
model = build_model(height, width, channels, actions)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 4, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 24, 18, 64)     32832     
_________________________________________________________________
flatten_2 (Flatten)          (None, 110592)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               28311808  
_________________________________________________________________
dense_6 (Dense)              (None, 9)                 2313      
Total params: 28,353,129
Trainable params: 28,353,129
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [11]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=10000)
    memory = SequentialMemory(limit=1000000, window_length=4) #limit = 100 000
    dqn = DQNAgent(
        batch_size=128,
        model=model, 
        memory=memory, 
        policy=policy, 
        enable_dueling_network=True, 
        dueling_type='avg', 
        nb_actions=actions,
        nb_steps_warmup=1000
    )
    return dqn

In [22]:
del model
model = build_model(height, width, channels, actions)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 4, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 24, 18, 64)     32832     
_________________________________________________________________
flatten_3 (Flatten)          (None, 110592)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               28311808  
_________________________________________________________________
dense_8 (Dense)              (None, 9)                 2313      
Total params: 28,353,129
Trainable params: 28,353,129
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))

In [14]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [15]:
tf.debugging.set_log_device_placement(True)

In [16]:
with tf.device('/GPU:0'):
    dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
done, took 4.283 seconds


In [27]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 70.000, steps: 691


KeyboardInterrupt: 

In [24]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

[TIP] Next time specify overwrite=True!


In [25]:
del model


https://stable-baselines3.readthedocs.io/en/master/common/atari_wrappers.html  
https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf  
https://www.nature.com/articles/nature14236  
### GPU
### PreProces
### Mniejsza sieć

In [26]:
dqn.load_weights('SavedWeights/10k-Fast/dqn_weights.h5f')